In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import tensorflow_addons as tfa

In [2]:
# Path to train images
train_dir = '../input/plant-seedlings-classification/train/'
train_dir_seg = '../input/plant-seedling-segmented/plant-seedling-segmented/seg_train/'
train_dir_large = '../input/plantseedlingslarge/plant-seedlings-large/train-large/'
train_dir_seg_large = '../input/plantseedlingslarge/plant-seedlings-large/train-large-seg/'


# Path to test images
test_dir = '../input/plant-seedlings-classification/'
test_dir_seg = '../input/plant-seedling-segmented/plant-seedling-segmented/'

In [3]:
train = train_dir
test = test_dir

In [4]:
nb_epoch     = 100
batch_size   = 16
width        = 299
height       = 299

In [5]:
species_list = ["Black-grass", "Charlock", "Cleavers", "Common Chickweed", "Common wheat", "Fat Hen",
                 "Loose Silky-bent", "Maize", "Scentless Mayweed", "Shepherds Purse", "Small-flowered Cranesbill",
                "Sugar beet"]

In [6]:
def define_generators():
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.inception_resnet_v2.preprocess_input,
        rotation_range=360,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.5,
        vertical_flip=True,
        horizontal_flip=True,
        validation_split=0.0,
    )

    train_generator = train_datagen.flow_from_directory(
        directory=train,
        target_size=(width, height),
        batch_size=batch_size,
        color_mode='rgb',
        class_mode="categorical",
        subset='training',
    )

    validation_generator = train_datagen.flow_from_directory(
        directory=train,
        target_size=(width, height),
        batch_size=batch_size,
        color_mode='rgb',
        class_mode="categorical",
        subset='validation',
        shuffle=False
    )

    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.inception_resnet_v2.preprocess_input)

    test_generator = test_datagen.flow_from_directory(
        directory=test,
        classes=['test'],
        target_size=(width, height),
        batch_size=1,
        color_mode='rgb',
        shuffle=False,
        class_mode='categorical')

    return train_generator, validation_generator, test_generator

In [7]:
# define appropriate callbacks
def training_callbacks():
    
    # save best model regularly
    save_best_model = tf.keras.callbacks.ModelCheckpoint(filepath = 'model.h5',
        monitor = 'accuracy', save_best_only = True, verbose = 1)
    
    # reduce learning rate when it stops decreasing
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'loss', factor = 0.4,
                              patience = 3, min_lr = 1e-10, verbose = 1, cooldown = 1)
    
    # stop training early if no further improvement
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'loss', min_delta = 1e-2, patience = 15, verbose = 1,
        mode = 'min', baseline = None, restore_best_weights = True
    )

    return save_best_model, reduce_lr, early_stopping


In [8]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten
import tensorflow_addons as tfa

def create_model():
    
    model_input = tf.keras.layers.Input(shape=(width, height, 3), name='image_input')
    model_main = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(input_shape=(width, height, 3), include_top=False, weights='imagenet')(model_input)
            
    model_dense1 = GlobalAveragePooling2D(name="avg_pool")(model_main)
    model_dense1 = BatchNormalization()(model_dense1)

    model_dense2 = tf.keras.layers.Dense(256, activation =  tfa.activations.mish, activity_regularizer=tf.keras.regularizers.l2(1e-5))(model_dense1)
    model_dense2 = BatchNormalization()(model_dense2)
    dropout_2 = tf.keras.layers.Dropout(0.25)(model_dense2)
    model_out = tf.keras.layers.Dense(12, activation="softmax")(dropout_2)

    model = tf.keras.models.Model(model_input,  model_out)
    optimizer = tf.keras.optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [9]:
model = create_model()
model.summary()

2022-04-10 15:43:29.922084: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-10 15:43:30.072280: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-10 15:43:30.073613: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-10 15:43:30.075791: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

219070464/219055592 [==============================] - 2s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, 299, 299, 3)]     0         
_________________________________________________________________
inception_resnet_v2 (Functio (None, 8, 8, 1536)        54336736  
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 1536)              0         
_________________________________________________________________
batch_normalization_203 (Bat (None, 1536)              6144      
_________________________________________________________________
dense (Dense)                (None, 256)               393472    
_________________________________________________________________
batch_normalization_204 (Bat (None, 256)               1024      
____________________________________________________________

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [10]:
train_generator, validation_generator, test_generator = define_generators()

history = model.fit(
    train_generator,
    epochs=nb_epoch,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data= validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks = training_callbacks()
)

Found 4750 images belonging to 12 classes.
Found 0 images belonging to 12 classes.
Found 794 images belonging to 1 classes.


2022-04-10 15:43:44.750848: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2022-04-10 15:44:08.160950: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


296/296 [==============================] - 282s 852ms/step - loss: 1.2512 - accuracy: 0.6113

Epoch 00001: accuracy improved from -inf to 0.61132, saving model to model.h5
Epoch 2/100
296/296 [==============================] - 201s 676ms/step - loss: 0.4600 - accuracy: 0.8483

Epoch 00002: accuracy improved from 0.61132 to 0.84833, saving model to model.h5
Epoch 3/100
296/296 [==============================] - 199s 669ms/step - loss: 0.3366 - accuracy: 0.8895

Epoch 00003: accuracy improved from 0.84833 to 0.88952, saving model to model.h5
Epoch 4/100
296/296 [==============================] - 199s 671ms/step - loss: 0.2865 - accuracy: 0.9045

Epoch 00004: accuracy improved from 0.88952 to 0.90452, saving model to model.h5
Epoch 5/100
296/296 [==============================] - 199s 671ms/step - loss: 0.2354 - accuracy: 0.9221

Epoch 00005: accuracy improved from 0.90452 to 0.92205, saving model to model.h5
Epoch 6/100
296/296 [==============================] - 201s 679ms/step - loss: 0

In [11]:
# plt.plot(history.history['loss'], label='train')
# #plt.plot(history.history['val_loss'], label = "validation")
# plt.legend(loc='upper right')
# plt.title('Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.show()
# plt.savefig('Loss.png')

In [12]:
# plt.plot(history.history['accuracy'], label='train')
# #plt.plot(history.history['val_accuracy'], label = "validation")
# plt.legend(loc='upper left')
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.show()
# plt.savefig('Accuracy.png')

In [13]:
model.save_weights('inception_resnet_v2.h5')

In [14]:
# validation_generator.reset() 
# val_pred = model.predict(validation_generator, steps=validation_generator.samples)
# print(val_pred.shape)
# y_pred = np.argmax(val_pred, axis = 1)
# y_true = validation_generator.classes

# cm = confusion_matrix(y_true, y_pred)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=species_list)

# disp.plot(cmap=plt.cm.Blues)
# plt.xticks(rotation=90)
# plt.show()

In [15]:
predictions = model.predict(test_generator, steps=test_generator.samples)

class_list = []

for i in range(0, predictions.shape[0]):
    y_class = predictions[i, :].argmax(axis=-1)
    class_list += [species_list[y_class]]

submission = pd.DataFrame()
submission['file'] = test_generator.filenames
submission['file'] = submission['file'].str.replace(r'test/', '')
submission['species'] = class_list

submission.to_csv('inception_resnet_v2_submission.csv', index=False)

print('Submission file generated. All done.')

Submission file generated. All done.
